In [ ]:
# Module Importations

In [ ]:
# Project Module Importations

In [ ]:

    # Make predictions on the test data.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    print("mlp nn MAE: " + str(mae))

    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print("mlp nn MSE: " + str(rmse))